**AUTORES**
* Mauro Barquinero
* Yandri Uchuari
* Marck Murillo
* Matias Tripode

# Trabajo Práctico Final: Linear/Quadratic Discriminant Analysis (LDA/QDA)

### Definición: Clasificador Bayesiano

Sean $k$ poblaciones, $x \in \mathbb{R}^p$ puede pertenecer a cualquiera $g \in \mathcal{G}$ de ellas. Bajo un esquema bayesiano, se define entonces $\pi_j \doteq P(G = j)$ la probabilidad *a priori* de que $X$ pertenezca a la clase *j*, y se **asume conocida** la distribución condicional de cada observable dado su clase $f_j \doteq f_{X|G=j}$.

De esta manera dicha probabilidad *a posteriori* resulta
$$
P(G|_{X=x} = j) = \frac{f_{X|G=j}(x) \cdot p_G(j)}{f_X(x)} \propto f_j(x) \cdot \pi_j
$$

La regla de decisión de Bayes es entonces
$$
H(x) \doteq \arg \max_{g \in \mathcal{G}} \{ P(G|_{X=x} = j) \} = \arg \max_{g \in \mathcal{G}} \{ f_j(x) \cdot \pi_j \}
$$

es decir, se predice a $x$ como perteneciente a la población $j$ cuya probabilidad a posteriori es máxima.

*Ojo, a no desesperar! $\pi_j$ no es otra cosa que una constante prefijada, y $f_j$ es, en su esencia, un campo escalar de $x$ a simplemente evaluar.*

### Distribución condicional

Para los clasificadores de discriminante cuadrático y lineal (QDA/LDA) se asume que $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma_j)$, es decir, se asume que cada población sigue una distribución normal.

Por definición, se tiene entonces que para una clase $j$:
$$
f_j(x) = \frac{1}{(2 \pi)^\frac{p}{2} \cdot |\Sigma_j|^\frac{1}{2}} e^{- \frac{1}{2}(x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j)}
$$

Aplicando logaritmo (que al ser una función estrictamente creciente no afecta el cálculo de máximos/mínimos), queda algo mucho más práctico de trabajar:

$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$

Observar que en este caso $C=-\frac{p}{2} \log(2\pi)$, pero no se tiene en cuenta ya que al tener una constante aditiva en todas las clases, no afecta al cálculo del máximo.

### LDA

En el caso de LDA se hace una suposición extra, que es $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma)$, es decir que las poblaciones no sólo siguen una distribución normal sino que son de igual matriz de covarianzas. Reemplazando arriba se obtiene entonces:

$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

Ahora, como $-\frac{1}{2}\log |\Sigma|$ es común a todas las clases se puede incorporar a la constante aditiva y, distribuyendo y reagrupando términos sobre $(x-\mu_j)^T \Sigma^{-1} (x- \mu_j)$ se obtiene finalmente:

$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$

### Entrenamiento/Ajuste

Obsérvese que para ambos modelos, ajustarlos a los datos implica estimar los parámetros $(\mu_j, \Sigma_j) \; \forall j = 1, \dots, k$ en el caso de QDA, y $(\mu_j, \Sigma)$ para LDA.

Estos parámetros se estiman por máxima verosimilitud, de manera que los estimadores resultan:

* $\hat{\mu}_j = \bar{x}_j$ el promedio de los $x$ de la clase *j*
* $\hat{\Sigma}_j = s^2_j$ la matriz de covarianzas estimada para cada clase *j*
* $\hat{\pi}_j = f_{R_j} = \frac{n_j}{n}$ la frecuencia relativa de la clase *j* en la muestra
* $\hat{\Sigma} = \frac{1}{n} \sum_{j=1}^k n_j \cdot s^2_j$ el promedio ponderado (por frecs. relativas) de las matrices de covarianzas de todas las clases. *Observar que se utiliza el estimador de MV y no el insesgado*

Es importante notar que si bien todos los $\mu, \Sigma$ deben ser estimados, la distribución *a priori* puede no inferirse de los datos sino asumirse previamente, utilizándose como entrada del modelo.

### Predicción

Para estos modelos, al igual que para cualquier clasificador Bayesiano del tipo antes visto, la estimación de la clase es por método *plug-in* sobre la regla de decisión $H(x)$, es decir devolver la clase que maximiza $\hat{f}_j(x) \cdot \hat{\pi}_j$, o lo que es lo mismo $\log\hat{f}_j(x) + \log\hat{\pi}_j$.

## Estructura del código

## Modelo

In [265]:
import numpy as np
from numpy.linalg import det, inv

**NOTAS**

**Clase ClassEncoder**
* Convierte las etiquetas de las clases en números enteros para facilitar los cálculos.
Proporciona un mapeo entre las etiquetas originales de las clases y los números enteros codificados, lo que permite que las predicciones se devuelvan en su forma original.


In [266]:
class ClassEncoder:
  def fit(self, y):
    self.names = np.unique(y)
    self.name_to_class = {name:idx for idx, name in enumerate(self.names)}
    self.fmt = y.dtype
    # Q1: por que no hace falta definir un class_to_name para el mapeo inverso?

  def _map_reshape(self, f, arr):
    return np.array([f(elem) for elem in arr.flatten()]).reshape(arr.shape)
    # Q2: por que hace falta un reshape?

  def transform(self, y):
    return self._map_reshape(lambda name: self.name_to_class[name], y)

  def fit_transform(self, y):
    self.fit(y)
    return self.transform(y)

  def detransform(self, y_hat):
    return self._map_reshape(lambda idx: self.names[idx], y_hat)

In [267]:
class BaseBayesianClassifier:
  def __init__(self):
    self.encoder = ClassEncoder()

  def _estimate_a_priori(self, y):
    a_priori = np.bincount(y.flatten().astype(int)) / y.size
    # Q3: para que sirve bincount?
    # R3: np.bincount cuenta la cantidad de ocurrencias de cada valor entero en un arreglo unidimensional. 
    # En el contexto del método _estimate_a_priori, se utiliza para contar cuántas muestras pertenecen a cada clase, 
    # dado que las clases han sido previamente codificadas como enteros (por ejemplo, 0, 1, 2, etc.).
    return np.log(a_priori)

  def _fit_params(self, X, y):
    # estimate all needed parameters for given model
    raise NotImplementedError()

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    raise NotImplementedError()

  def fit(self, X, y, a_priori=None):
    # first encode the classes
    y = self.encoder.fit_transform(y)

    # if it's needed, estimate a priori probabilities
    self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)

    # check that a_priori has the correct number of classes
    assert len(self.log_a_priori) == len(self.encoder.names), "A priori probabilities do not match number of classes"

    # now that everything else is in place, estimate all needed parameters for given model
    self._fit_params(X, y)
    # Q4: por que el _fit_params va al final? no se puede mover a, por ejemplo, antes de la priori?
    # R4: La función _fit_params se ejecuta después de calcular las probabilidades a priori porque:
    # - Se necesita que las clases ya estén codificadas en enteros mediante self.encoder.fit_transform(y). Esto asegura que cada clase esté correctamente mapeada para los cálculos, como medias y covarianzas.
    # - Antes de calcular parámetros como las medias y matrices de covarianza por clase, es necesario verificar que el número de clases coincida con las probabilidades a priori estimadas. Este chequeo se realiza justo antes de _fit_params.

  def predict(self, X):
    # this is actually an individual prediction encased in a for-loop
    m_obs = X.shape[1]
    y_hat = np.empty(m_obs, dtype=self.encoder.fmt)

    for i in range(m_obs):
      encoded_y_hat_i = self._predict_one(X[:,i].reshape(-1,1))
      y_hat[i] = self.encoder.names[encoded_y_hat_i]

    # return prediction as a row vector (matching y)
    return y_hat.reshape(1,-1)

  def _predict_one(self, x):
    # calculate all log posteriori probabilities (actually, +C)
    log_posteriori = [ log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i
                  in enumerate(self.log_a_priori) ]

    # return the class that has maximum a posteriori probability
    return np.argmax(log_posteriori)

In [268]:
class QDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    # estimate each covariance matrix
    self.inv_covs = [inv(np.cov(X[:,y.flatten()==idx], bias=True))
                      for idx in range(len(self.log_a_priori))]
    # Q5: por que hace falta el flatten y no se puede directamente X[:,y==idx]?
    # R5: Asegura que y sea un arreglo unidimensional para facilitar el indexado y las comparaciones.

    # Q6: por que se usa bias=True en vez del default bias=False?
    # R6: El bias=True en np.cov: Se alinea con el supuesto de que los datos provienen de una 
    # distribución gaussiana, donde las matrices de covarianza se normalizan por el número total de muestras.
    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]
    # Q7: que hace axis=1? por que no axis=0?
    # R7: Calcula la media a lo largo de las muestras para cada característica (columna).

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

In [269]:
class TensorizedQDA(QDA):

    def _fit_params(self, X, y):
        # ask plain QDA to fit params
        super()._fit_params(X,y)

        # stack onto new dimension
        self.tensor_inv_cov = np.stack(self.inv_covs)
        self.tensor_means = np.stack(self.means)

    def _predict_log_conditionals(self,x):
        unbiased_x = x - self.tensor_means
        inner_prod = unbiased_x.transpose(0,2,1) @ self.tensor_inv_cov @ unbiased_x

        return 0.5*np.log(det(self.tensor_inv_cov)) - 0.5 * inner_prod.flatten()

    def _predict_one(self, x):
        # return the class that has maximum a posteriori probability
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x))

## Código para pruebas

Seteamos los datos

In [270]:
# hiperparámetros
rng_seed = 6543

In [271]:
from sklearn.datasets import load_iris, fetch_openml

def get_iris_dataset():
  data = load_iris()
  X_full = data.data
  y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
  return X_full, y_full

def get_penguins():
    # get data
    df, tgt = fetch_openml(name="penguins", return_X_y=True, as_frame=True)

    # drop non-numeric columns
    df.drop(columns=["island","sex"], inplace=True)

    # drop rows with missing values
    mask = df.isna().sum(axis=1) == 0
    df = df[mask]
    tgt = tgt[mask]

    return df.values, tgt.to_numpy().reshape(-1,1)

# showing for iris
X_full, y_full = get_iris_dataset()

print(f"X: {X_full.shape}, Y:{y_full.shape}")

X: (150, 4), Y:(150, 1)


In [272]:
# peek data matrix
X_full[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [273]:
# peek target vector
y_full[:5]

array([['setosa'],
       ['setosa'],
       ['setosa'],
       ['setosa'],
       ['setosa']], dtype='<U10')

Separamos el dataset en train y test para medir performance

In [274]:
# preparing data, train - test validation
# 70-30 split
from sklearn.model_selection import train_test_split
# NOTE: los parametros X, y, test_sz, random_state no se estaban utilizando. Corregimos el codigo.
def split_transpose(X, y, test_sz, random_state):
    # split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_sz, random_state=random_state)

    # transpose so observations are column vectors
    return X_train.T, y_train.T, X_test.T, y_test.T

def accuracy(y_true, y_pred):
  return (y_true == y_pred).mean()

train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, 0.4, rng_seed)

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(4, 90) (1, 90) (4, 60) (1, 60)


Entrenamos un QDA y medimos su accuracy

In [275]:
qda = QDA()

qda.fit(train_x, train_y)

In [276]:
train_acc = accuracy(train_y, qda.predict(train_x))
test_acc = accuracy(test_y, qda.predict(test_x))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0111 while test error is 0.0167


Con el magic %%timeit podemos estimar el tiempo que tarda en correr una celda en base a varias ejecuciones. Por poner un ejemplo, acá vamos a estimar lo que tarda un ciclo completo de QDA y también su inferencia (predicción).

Ojo! a veces [puede ser necesario ejecutarlo varias veces](https://stackoverflow.com/questions/10994405/python-timeit-results-cached-instead-of-calculated) para obtener resultados consistentes.

In [277]:
%%timeit

qda.predict(test_x)

1.24 ms ± 43.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [278]:
%%timeit

model = QDA()
model.fit(train_x, train_y)
model.predict(test_x)

1.34 ms ± 10.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Consigna

## Implementación
1. Entrenar un modelo QDA sobre el dataset *iris* utilizando las distribuciones *a priori* a continuación ¿Se observan diferencias?¿Por qué cree? _Pista: comparar con las distribuciones del dataset completo, **sin splitear**_.
    1. Uniforme (cada clase tiene probabilidad 1/3)
    2. Una clase con probabilidad 0.9, las demás 0.05 (probar las 3 combinaciones)
2. Repetir el punto anterior para el dataset *penguin*.
3. Implementar el modelo LDA, entrenarlo y testearlo contra los mismos sets que QDA ¿Se observan diferencias? ¿Podría decirse que alguno de los dos es notoriamente mejor que el otro?
4. Utilizar otros 2 (dos) valores de *random seed* para obtener distintos splits de train y test, y repetir la comparación del punto anterior ¿Las conclusiones previas se mantienen?
5. Estimar y comparar los tiempos de predicción de las clases `QDA` y `TensorizedQDA`. De haber diferencias ¿Cuáles pueden ser las causas?


**Sugerencia:** puede resultar de utilidad para los puntos de comparación utilizar tablas del siguiente estilo:

<center>

Modelo | Dataset | Seed | Error (train) | Error (test)
:---: | :---: | :---: | :---: | :---:
QDA | Iris | 125 | 0.55 | 0.85
LDA | Iris | 125 | 0.22 | 0.8

</center>

## Preguntas teóricas

1. En LDA se menciona que la función a maximizar puede ser, mediante operaciones, convertida en:
$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$
Mostrar los pasos por los cuales se llega a dicha expresión.
2. Explicar, utilizando las respectivas funciones a maximizar, por qué QDA y LDA son "quadratic" y "linear".
3. La implementación de QDA estima la probabilidad condicional utilizando `0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x` que no es *exactamente* lo descrito en el apartado teórico ¿Cuáles son las diferencias y por qué son expresiones equivalentes?

El espíritu de esta componente práctica es la de establecer un mínimo de trabajo aceptable para su entrega; se invita al alumno a explorar otros aspectos que generen curiosidad, sin sentirse de ninguna manera limitado por la consigna.

## Ejercicio teórico

Sea una red neuronal de dos capas, la primera de 3 neuronas y la segunda de 1 con los parámetros inicializados con los siguientes valores:
$$
w^{(1)} =
\begin{pmatrix}
0.1 & -0.5 \\
-0.3 & -0.9 \\
0.8 & 0.02
\end{pmatrix},
b^{(1)} = \begin{pmatrix}
0.1 \\
0.5 \\
0.8
\end{pmatrix},
w^{(2)} =
\begin{pmatrix}
-0.4 & 0.2 & -0.5
\end{pmatrix},
b^{(2)} = 0.7
$$

y donde cada capa calcula su salida vía

$$
y^{(i)} = \sigma (w^{(i)} \cdot x^{(i)}+b^{(i)})
$$

donde $\sigma (z) = \frac{1}{1+e^{-z}}$ es la función sigmoidea .

\\
Dada la observación $x=\begin{pmatrix}
1.8 \\
-3.4
\end{pmatrix}$, $y=5$ y la función de costo $J(\theta)=\frac{1}{2}(\hat{y}_\theta-y)^2$, calcular las derivadas de J respecto de cada parámetro $w^{(1)}$, $w^{(2)}$, $b^{(1)}$, $b^{(2)}$.

*Nota: Con una sigmoidea a la salida jamás va a poder estimar el 5 "pedido", pero eso no afecta al mecanismo de backpropagation!*

## Preguntas en el código
Previamente las preguntas "técnicas" en comentarios en el código eran parte del TP, y buscaban que el alumno logre entrar en el detalle de por qué cada linea de código es como es y en el orden en el que está. Ya no forman parte de la consigna, pero se aconseja al alumno intentar responderlas. Las respuestas a las mismas se encuentran en un archivo separado.

## Opcional

### QDA

Debido a la forma cuadrática de QDA, no se puede predecir para *n* observaciones en una sola pasada (utilizar $X \in \mathbb{R}^{p \times n}$ en vez de $x \in \mathbb{R}^p$) sin pasar por una matriz de *n x n* en donde se computan todas las interacciones entre observaciones. Se puede acceder al resultado recuperando sólo la diagonal de dicha matriz, pero resulta ineficiente en tiempo y (especialmente) en memoria. Aún así, es *posible* que el modelo funcione más rápido.

1. Implementar el modelo `FasterQDA` (se recomienda heredarlo de TensorizedQDA) de manera de eliminar el ciclo for en el método predict.
2. Comparar los tiempos de predicción de `FasterQDA` con `TensorizedQDA` y `QDA`
3. Mostrar (puede ser con un print) dónde aparece la mencionada matriz de *n x n*, donde *n* es la cantidad de observaciones a predecir.
4.Demostrar
$$
diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
$$ es decir, que se puede "esquivar" la matriz de *n x n* usando matrices de *n x p*.
5.Utilizar la propiedad antes demostrada para reimplementar la predicción del modelo `FasterQDA` de forma eficiente. ¿Hay cambios en los tiempos de predicción?


### LDA

1. "Tensorizar" el modelo LDA y comparar sus tiempos de predicción con el modelo antes implementado. *Notar que, en modo tensorizado, se puede directamente precomputar $\mu^T \cdot \Sigma^{-1} \in \mathbb{R}^{k \times 1 \times p}$ y guardar eso en vez de $\Sigma^{-1}$.*
2. LDA no sufre del problema antes descrito de QDA debido a que no computa productos internos, por lo que no tiene un verdadero costo extra en memoria predecir "en batch". Implementar el modelo `FasterLDA` y comparar sus tiempos de predicción con las versiones anteriores de LDA.

---------------------------------

# RESPUESTAS

1. Entrenar un modelo QDA sobre el dataset *iris* utilizando las distribuciones *a priori* a continuación ¿Se observan diferencias?¿Por qué cree? _Pista: comparar con las distribuciones del dataset completo, **sin splitear**_.
    1. Uniforme (cada clase tiene probabilidad 1/3)
    2. Una clase con probabilidad 0.9, las demás 0.05 (probar las 3 combinaciones)

* 1.1 Entrenar un modelo QDA sobre el dataset *iris* utilizando las distribuciones *a priori* Uniforme (cada clase tiene probabilidad 1/3)

**CODIGO DE SETUP**

El siguiente codigo de setup se utiliza para **todas las pruebas** (tanto QDA como LDA).
Simplemente se descomentan las lineas para el dataset que queremos utilizar
ejemplo: Si queremos utilizar el penguin quedaria
```
    # Cargar el dataset iris
    # X_full, y_full = get_iris_dataset()
    # Cargar el dataset penguin
    X_full, y_full = get_penguins()
```

Si queremos probar sin split quedaria:
```
    # [Split]: Dividir el dataset entre test y train 
    # train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, test_sz, seed)
    # [Sin Split]: Dataset completo sin split
    train_x, train_y, test_x, test_y = X_full.T, y_full.T, X_full.T, y_full.T
```


In [279]:
# *** CODIGO DE SETUP *** 

# Cargar el dataset iris
X_full, y_full = get_iris_dataset()
# Cargar el dataset penguin
# X_full, y_full = get_penguins()

# Seed
seed = 16
# Test size
test_sz = 0.3 # 30%
# [Split]: Dividir el dataset entre test y train 
train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, test_sz, seed)
# [Sin Split]: Dataset completo sin split
# train_x, train_y, test_x, test_y = X_full.T, y_full.T, X_full.T, y_full.T
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)


(4, 105) (1, 105) (4, 45) (1, 45)


**Uniforme (cada clase tiene probabilidad 1/3)**

In [280]:
qda = QDA()
qda.fit(train_x, train_y, a_priori=[0.33333333, 0.33333333,  0.33333333])

y_train_pred = qda.predict(train_x)
y_test_pred = qda.predict(test_x)
qda_train_acc = accuracy(train_y, y_train_pred)
qda_test_acc = accuracy(test_y, y_test_pred)
print(f"* QDA: Train (apparent) error is {1-qda_train_acc:.4f} while test error is {1-qda_test_acc:.4f}")

# Calculate train and test errors
error_train = np.mean(y_train_pred != train_y)
error_test = np.mean(y_test_pred != test_y)

print(f"Error (train): {error_train:.2%}")
print(f"Error (test): {error_test:.2%}")

* QDA: Train (apparent) error is 0.0381 while test error is 0.0222
Error (train): 3.81%
Error (test): 2.22%


* 1.2 Entrenar un modelo QDA sobre el dataset *iris* 
utilizando las distribuciones *a priori* Una clase con probabilidad 0.9, las demás 0.05 (probar las 3 combinaciones)


**Primera clase con probabilidad 0.9, las demás 0.05**

In [281]:
qda = QDA()
# Primera clase con probabilidad 0.9, las demás 0.05
qda.fit(train_x, train_y, a_priori=[0.9, 0.05,  0.05])

y_train_pred = qda.predict(train_x)
y_test_pred = qda.predict(test_x)
qda_train_acc = accuracy(train_y, y_train_pred)
qda_test_acc = accuracy(test_y, y_test_pred)
print(f"* QDA: Train (apparent) error is {1-qda_train_acc:.4f} while test error is {1-qda_test_acc:.4f}")

# Calculate train and test errors
error_train = np.mean(y_train_pred != train_y)
error_test = np.mean(y_test_pred != test_y)

print(f"Error (train): {error_train:.2%}")
print(f"Error (test): {error_test:.2%}")


* QDA: Train (apparent) error is 0.0381 while test error is 0.0222
Error (train): 3.81%
Error (test): 2.22%


**Segunda clase con probabilidad 0.05, 0.9 y 0.05**

In [282]:
qda = QDA()
# Segunda clase con probabilidad 0.05, 0.9 y 0.05
qda.fit(train_x, train_y, a_priori=[0.05, 0.9,  0.05])

y_train_pred = qda.predict(train_x)
y_test_pred = qda.predict(test_x)
qda_train_acc = accuracy(train_y, y_train_pred)
qda_test_acc = accuracy(test_y, y_test_pred)
print(f"* QDA: Train (apparent) error is {1-qda_train_acc:.4f} while test error is {1-qda_test_acc:.4f}")

# Calculate train and test errors
error_train = np.mean(y_train_pred != train_y)
error_test = np.mean(y_test_pred != test_y)

print(f"Error (train): {error_train:.2%}")
print(f"Error (test): {error_test:.2%}")


* QDA: Train (apparent) error is 0.0381 while test error is 0.0444
Error (train): 3.81%
Error (test): 4.44%


**Tercera clase con probabilidad 0.05, 0.05 y 0.9**

In [283]:
qda = QDA()
# Tercera clase con probabilidad 0.05, 0.05 y 0.9
qda.fit(train_x, train_y, a_priori=[0.05, 0.05,  0.9])

y_train_pred = qda.predict(train_x)
y_test_pred = qda.predict(test_x)
qda_train_acc = accuracy(train_y, y_train_pred)
qda_test_acc = accuracy(test_y, y_test_pred)
print(f"* QDA: Train (apparent) error is {1-qda_train_acc:.4f} while test error is {1-qda_test_acc:.4f}")

# Calculate train and test errors
error_train = np.mean(y_train_pred != train_y)
error_test = np.mean(y_test_pred != test_y)

print(f"Error (train): {error_train:.2%}")
print(f"Error (test): {error_test:.2%}")

* QDA: Train (apparent) error is 0.0286 while test error is 0.0667
Error (train): 2.86%
Error (test): 6.67%


Modelo | Dataset | Seed | a_priori | Split | Error (train) | Error (test)
:---: | :---: | :---: | :---: | :---: | :---: | :---:
QDA | Iris | 125 | [1/3, 1/3, 1/3] | 70/30 | 2.86% | 0.00%
QDA | Iris | 125 | [0.9, 0.05, 0.05] | 70/30 | 2.86% | 0.00%
QDA | Iris | 125 | [0.05, 0.9, 0.05] | 70/30 | 6.67% | 2.22%
QDA | Iris | 125 | [0.05, 0.05, 0.9] | 70/30 | 4.76% | 0.00%
QDA | Iris | 125 | [1/3, 1/3, 1/3] | 100 | 2.00% | 2.00%
QDA | Iris | 125 | [0.9, 0.05, 0.05] | 100 | 2.00% | 2.00%
QDA | Iris | 125 | [0.05, 0.9, 0.05] | 100 | 3.33% | 3.33%
QDA | Iris | 125 | [0.05, 0.05, 0.9] | 100 | 4.00% | 4.00%
 |  |  |  |  |  |
QDA | Penguin | 125 | [1/3, 1/3, 1/3] | 70/30 | 0.84% | 0.97%
QDA | Penguin | 125 | [0.9, 0.05, 0.05] | 70/30 | 1.67% | 1.94%
QDA | Penguin | 125 | [0.05, 0.9, 0.05] | 70/30 | 4.60% | 6.80%
QDA | Penguin | 125 | [0.05, 0.05, 0.9] | 70/30 | 0.84% | 0.97%
QDA | Penguin | 125 | [1/3, 1/3, 1/3] | 100 | 0.88% | 0.88%
QDA | Penguin | 125 | [0.9, 0.05, 0.05] | 100 | 1.75% | 1.75%
QDA | Penguin | 125 | [0.05, 0.9, 0.05] | 100 | 3.51% | 3.51%
QDA | Penguin | 125 | [0.05, 0.05, 0.9] | 100 | 0.88% | 0.88%



------------------------

3. Implementar el modelo **LDA**, entrenarlo y testearlo contra los mismos sets que QDA 
* ¿Se observan diferencias?
* ¿Podría decirse que alguno de los dos es notoriamente mejor que el otro?

3. Implementation LDA

In [284]:
# [Analisis Discriminante Lineal y Cuadratico](https://www.youtube.com/watch?v=Meo0odkRFfU&list=PLN2e9R_DoC0SDbv3C_B-LCpuliGtvdQvg&index=1&ab_channel=AndresFarall)
class LDA(BaseBayesianClassifier):
    """
    Este método calcula y almacena los parámetros necesarios para el modelo. Es el paso de "entrenamiento" o ajuste del modelo.
    """
    def _fit_params(self, X, y):
        # Asegura que y (el vector de etiquetas) sea un arreglo unidimensional. Esto facilita las comparaciones e indexaciones más adelante.
        # Ejemplo: Si y es una matriz columna de forma (n,1), después del flatten, será un vector de forma (n,).
        y_flat = y.flatten()
        # Calcula la matriz de covarianza compartida (Σ) para todas las clases combinadas.
        # bias=True: Normaliza por n (número de muestras) en lugar de n − 1. Esto se alinea con los supuestos teóricos del modelo LDA.
        self.shared_cov = np.cov(X, bias=True)
        # Calcula la inversa de la matriz de covarianza compartida, que se usará posteriormente para evaluar las probabilidades condicionales.
        # La matriz inversa permite medir la "dispersión" de los datos considerando las correlaciones entre las características.
        self.inv_shared_cov = np.linalg.inv(self.shared_cov)    

        # Calcula los vectores de media (μj)​ para cada clase j
        # X[:, y_flat == idx] : Filtra las columnas de X correspondientes a la clase j (donde y = idx)
        # .mean(axis=1, keepdims=True): Calcula la media para cada fila (característica), manteniendo la forma de matriz (columna).
        self.means = [
            X[:, y_flat == idx].mean(axis=1, keepdims=True)
            for idx in range(len(self.log_a_priori))
        ]
        # self.means : Es una lista donde cada elemento es un vector columna (μj) de medias para cada clase.
    """
    Este método calcula el logaritmo de la probabilidad condicional para una observación x dada y una clase j.
    """
    def _predict_log_conditional(self, x, class_idx):
        # Resta el vector de medias (μj) correspondiente a la clase j de la observación x.
        # Esto genera un vector desplazado (x − μj) que mide qué tan lejos está x del centro de la clase j.
        unbiased_x = x - self.means[class_idx]
        # Este término mide qué tan probable es que x pertenezca a la clase j, considerando las correlaciones entre las características (por Σ^−1)
        return -0.5 * unbiased_x.T @ self.inv_shared_cov @ unbiased_x

* 3.1 Entrenar un modelo LDA sobre el dataset *iris* y *penguin* utilizando las distribuciones *a priori* Uniforme (cada clase tiene probabilidad 1/3)

**Uniforme (cada clase tiene probabilidad 1/3)**

In [285]:
lda = LDA()
lda.fit(train_x, train_y, a_priori=[0.33333333, 0.33333333,  0.33333333])

y_train_pred = lda.predict(train_x)
y_test_pred = lda.predict(test_x)
lda_train_acc = accuracy(train_y, y_train_pred)
lda_test_acc = accuracy(test_y, y_test_pred)
print(f"* LDA: Train (apparent) error is {1-lda_train_acc:.4f} while test error is {1-lda_test_acc:.4f}")

# Calculate train and test errors
error_train = np.mean(y_train_pred != train_y)
error_test = np.mean(y_test_pred != test_y)

print(f"Error (train): {error_train:.2%}")
print(f"Error (test): {error_test:.2%}")

* LDA: Train (apparent) error is 0.1048 while test error is 0.2000
Error (train): 10.48%
Error (test): 20.00%


* 3.2 Entrenar un modelo LDA sobre el dataset *iris* y *penguin*
utilizando las distribuciones *a priori* Una clase con probabilidad 0.9, las demás 0.05 (probar las 3 combinaciones)

**Primera clase con probabilidad 0.9, las demás 0.05**

In [286]:
lda = LDA()
# Primera clase con probabilidad 0.9, las demás 0.05
lda.fit(train_x, train_y, a_priori=[0.9, 0.05,  0.05])

y_train_pred = lda.predict(train_x)
y_test_pred = lda.predict(test_x)
lda_train_acc = accuracy(train_y, y_train_pred)
lda_test_acc = accuracy(test_y, y_test_pred)
print(f"* LDA: Train (apparent) error is {1-lda_train_acc:.4f} while test error is {1-lda_test_acc:.4f}")

# Calculate train and test errors
error_train = np.mean(y_train_pred != train_y)
error_test = np.mean(y_test_pred != test_y)

print(f"Error (train): {error_train:.2%}")
print(f"Error (test): {error_test:.2%}")

* LDA: Train (apparent) error is 0.5048 while test error is 0.5333
Error (train): 50.48%
Error (test): 53.33%


**Segunda clase con probabilidad 0.9, las demás 0.05**

In [287]:
lda = LDA()
# Segunda clase con probabilidad 0.9, las demás 0.05
lda.fit(train_x, train_y, a_priori=[0.05, 0.9,  0.05])

y_train_pred = lda.predict(train_x)
y_test_pred = lda.predict(test_x)
lda_train_acc = accuracy(train_y, y_train_pred)
lda_test_acc = accuracy(test_y, y_test_pred)
print(f"* LDA: Train (apparent) error is {1-lda_train_acc:.4f} while test error is {1-lda_test_acc:.4f}")

# Calculate train and test errors
error_train = np.mean(y_train_pred != train_y)
error_test = np.mean(y_test_pred != test_y)

print(f"Error (train): {error_train:.2%}")
print(f"Error (test): {error_test:.2%}")

* LDA: Train (apparent) error is 0.6286 while test error is 0.5778
Error (train): 62.86%
Error (test): 57.78%


**Tercera clase con probabilidad 0.9, las demás 0.05**

In [288]:
lda = LDA()
# Tercera clase con probabilidad 0.9, las demás 0.05
lda.fit(train_x, train_y, a_priori=[0.05, 0.05,  0.9])

y_train_pred = lda.predict(train_x)
y_test_pred = lda.predict(test_x)
lda_train_acc = accuracy(train_y, y_train_pred)
lda_test_acc = accuracy(test_y, y_test_pred)
print(f"* LDA: Train (apparent) error is {1-lda_train_acc:.4f} while test error is {1-lda_test_acc:.4f}")

# Calculate train and test errors
error_train = np.mean(y_train_pred != train_y)
error_test = np.mean(y_test_pred != test_y)

print(f"Error (train): {error_train:.2%}")
print(f"Error (test): {error_test:.2%}")

* LDA: Train (apparent) error is 0.5429 while test error is 0.5556
Error (train): 54.29%
Error (test): 55.56%


Modelo | Dataset | Seed | a_priori | Split | Error (train) | Error (test)
:---: | :---: | :---: | :---: | :---: | :---: | :---:
LDA | Iris | 125 | [1/3, 1/3, 1/3] | 70/30 | 12.38% | 13.33%
LDA | Iris | 125 | [0.9, 0.05, 0.05] | 70/30 | 54.29% | 46.67%
LDA | Iris | 125 | [0.05, 0.9, 0.05] | 70/30 | 63.81% | 55.56%
LDA | Iris | 125 | [0.05, 0.05, 0.9] | 70/30 | 53.33% | 48.89%
LDA | Iris | 125 | [1/3, 1/3, 1/3] | 100 | 13.33% | 13.33%
LDA | Iris | 125 | [0.9, 0.05, 0.05] | 100 | 51.33% | 51.33%
LDA | Iris | 125 | [0.05, 0.9, 0.05] | 100 | 62.67% | 62.67%
LDA | Iris | 125 | [0.05, 0.05, 0.9] | 100 | 55.33% | 55.33%
 |  |  |  |  |  | 
LDA | Penguin | 125 | [1/3, 1/3, 1/3] | 70/30 | 0.84% | 0.97% 
LDA | Penguin | 125 | [0.9, 0.05, 0.05] | 70/30 | 44.77% | 37.86%
LDA | Penguin | 125 | [0.05, 0.9, 0.05] | 70/30 | 38.49% | 44.66%
LDA | Penguin | 125 | [0.05, 0.05, 0.9] | 70/30 | 45.19% | 39.81%
LDA | Penguin | 125 | [1/3, 1/3, 1/3] | 100 | 0.88% | 0.88%
LDA | Penguin | 125 | [0.9, 0.05, 0.05] | 100 | 42.98% | 42.98%
LDA | Penguin | 125 | [0.05, 0.9, 0.05] | 100 | 40.64% | 40.64%
LDA | Penguin | 125 | [0.05, 0.05, 0.9] | 100 | 43.57% | 43.57%

* ¿Se observan diferencias?
    - Comparando ambas tablas vemos que el QDA produce errors (train y test) mas pequeños que los producidos por LDA.
* ¿Podría decirse que alguno de los dos es notoriamente mejor que el otro?
    - De la respuesta anterior podemos decir que QDA performa mejor que LDA.


-------------------

4. Utilizar otros 2 (dos) valores de *random seed* para obtener distintos splits de train y test, y repetir la comparación del punto anterior ¿Las conclusiones previas se mantienen?

Se realiza la comparacion con Seeds de 125, 64 y 16. En todos los casos el split es 70-30 t la distribucion a priori es 1/3 para todas las clases:

Modelo | Dataset | Seed | a_priori | Split | Error (train) | Error (test)
:---: | :---: | :---: | :---: | :---: | :---: | :---:
QDA | Iris | 125 | [1/3, 1/3, 1/3] | 70/30 | 2.86% | 0.00%
QDA | Iris | 64 | [1/3, 1/3, 1/3] | 70/30 | 0.00% | 4.44%   
QDA | Iris | 16 |  [1/3, 1/3, 1/3] | 70/30 | 3.81% |2.22%   
QDA | Penguin | 125 | [1/3, 1/3, 1/3] | 70/30 | 0.84% | 0.97%
QDA | Penguin | 64 | [1/3, 1/3, 1/3] | 70/30 | 0.84% | 1.94%
QDA | Penguin | 16 |  [1/3, 1/3, 1/3] | 70/30 | 1.26% | 0.97%
|  |  |  |  |  | 
LDA | Iris | 125 | [1/3, 1/3, 1/3] | 70/30 | 12.38% | 13.33%
LDA | Iris | 64 | [1/3, 1/3, 1/3] | 70/30 | 14.29% | 11.11% 
LDA | Iris | 16 |  [1/3, 1/3, 1/3] | 70/30 | 10.48% | 20.00%
LDA | Penguin | 125 | [1/3, 1/3, 1/3] | 70/30 | 0.84% | 0.97%
LDA | Penguin | 64 | [1/3, 1/3, 1/3] | 70/30 | 0.84% | 0.97%
LDA | Penguin | 16 |  [1/3, 1/3, 1/3] | 70/30 | 1.26% | 0.00%


¿Las conclusiones previas se mantienen?
- Vemos que las conclusiones se mantienen, siendo la performance QDA superior a la del LDA

------------------------------------------

5. Estimar y comparar los tiempos de predicción de las clases `QDA` y `TensorizedQDA`. De haber diferencias ¿Cuáles pueden ser las causas?

 Estimar los tiempos de predicción de `TensorizedQDA`

In [289]:

tensorized_qda = TensorizedQDA()
tensorized_qda.fit(train_x, train_y, a_priori=[0.33333333, 0.33333333,  0.33333333])




In [290]:
%%timeit
tensorized_qda.predict(train_x)

885 μs ± 52.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [291]:
qda = QDA()
qda.fit(train_x, train_y, a_priori=[0.33333333, 0.33333333,  0.33333333])


In [292]:
%%timeit
qda.predict(train_x)

2.16 ms ± 74.7 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Modelo | Dataset | Seed | a_priori | Split | Performance 
:---: | :---: | :---: | :---: | :---: | :---: 
Tensorized QDA | Penguins | 125 | [1/3, 1/3, 1/3] | 70/30 | 2.7 ms ± 112 μs 
QDA | Penguins | 125 | [1/3, 1/3, 1/3] | 70/30 | 7.23 ms ± 146 μs 

De haber diferencias ¿Cuáles pueden ser las causas?
- El Tensorized QDA performa ~2.7 veces mas rapido que el QDA y esto se debe a que cuenta con una implementacion mas de la funcion `_predict_log_conditionals`

Comparando como implementa cada modelo la funcion `_predict_log_conditionals`:
* `QDA` calcula la probabilidad condicional logarítmica para una clase a la vez, iterando sobre las clases para computar `x − μ` productos internos y determinantes.

* `TensorizedQDA` calcula `x − μ` productos internos y determinantes para todas las clases en un solo paso usando operaciones tensoriales:
    - `unbiased_x` = `x − tensor_means`: Hace un broadcast de x a través de todas las clases.
    - `inner_prod` = `unbiased_xT⋅tensor_inv_cov⋅unbiased_x`: Multiplicación tensorial eficiente a través de todas las clases.

------------------------------------------------

## Preguntas Teoricas


!['Pregunta 1'](pregunta_teorica_1.jpg) 

!['Pregunta 2'](pregunta_teorica_2.jpg) 

!['Pregunta 3'](pregunta_teorica_3.jpg) 

----------------------------------------------------------------------

## Ejercicio Teorico

!['Pregunta 1'](ejercicio_teorico_1.jpg) 

!['Pregunta 2'](ejercicio_teorico_2.jpg) 

!['Pregunta 3'](ejercicio_teorico_3.jpg) 